In [19]:
%%time
from multiprocessing import Pool
import numpy as np
p = Pool(10)
print(p.map(f, [1]*36))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
CPU times: user 283 ms, sys: 252 ms, total: 535 ms
Wall time: 16.2 s


In [1]:
load_ext cython

In [7]:
%%cython

import numpy as np
cimport numpy as np
def f(x):
    cdef np.ndarray first,second
    first, second= np.random.normal(0,1,[2000,2000]),np.random.normal(0,1,[2000,2000])
    np.dot(first,second)
    return 0